# Quickstart

```{include} warning.md

```

:::{note}
For installation instructions, please refer to the [installation guide](./installation).
:::



An agent is a software entity that communicates via messages, maintains its own state, and performs actions in response to received messages or changes in its state. 

In AgentChat, agents can be rapidly implemented using preset agent configurations. To illustrate this, we will begin with creating an agent that can address tasks by responding to messages it receives. 

In [1]:
import logging

from autogen_agentchat import EVENT_LOGGER_NAME
from autogen_agentchat.agents import CodingAssistantAgent, ToolUseAssistantAgent
from autogen_agentchat.logging import ConsoleLogHandler
from autogen_agentchat.teams import MaxMessageTermination, RoundRobinGroupChat
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.addHandler(ConsoleLogHandler())
logger.setLevel(logging.INFO)


# define a tool
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 73 degrees and Sunny."


# wrap the tool for use with the agent
get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

# define an agent
weather_agent = ToolUseAssistantAgent(
    name="writing_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-2024-08-06"),
    registered_tools=[get_weather_tool],
)

# add the agent to a team
agent_team = RoundRobinGroupChat([weather_agent])
result = await agent_team.run(
    task="What is the weather in New York?",
    termination_condition=MaxMessageTermination(max_messages=1),
)
print("\n", result)


--------------------------------------------------------------------------- 
[2024-10-20T09:01:32.381165]:

What is the weather in New York?
--------------------------------------------------------------------------- 
[2024-10-20T09:01:33.698359], writing_agent:

The weather in New York is currently 73 degrees and sunny. TERMINATE
--------------------------------------------------------------------------- 
[2024-10-20T09:01:33.698749], Termination:

Maximal number of messages 1 reached, current message count: 1
 TeamRunResult(messages=[TextMessage(source='user', content='What is the weather in New York?'), StopMessage(source='writing_agent', content='The weather in New York is currently 73 degrees and sunny. TERMINATE')])


The code snippet above introduces two high level concepts in AgentChat: `Agent` and `Team`.  An Agent helps us define what actions are taken when a message is received. Specifically, we use the `ToolUseAssistantAgent` preset - an agent that can be given a function that it can then use to address tasks.  A Team helps us define the rules for how agents interact with each other.  In the  `RoundRobinGroupChat` team, agents receive messages in a sequential round-robin fashion.  

## What's Next?

Now that you have a basic understanding of how to define an agent and a team, consider following the [tutorial](./tutorial/index) for a walkthrough on other features of AgentChat.

